<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Alternative Approaches to Expected Return

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Approaches

Topics of interest include:

* equal weights as a benchmark
* equal returns for all assets
 * resulting portfolio weights
 * out-of-sample performance 
* equal Sharpe ratio for all assets
 * resulting portfolio weights
 * out-of-sample performance
* CAPM-based return predictions
 * resulting portfolio weights
 * out-of-sample performance

## Real Financial Data

**_Historical end-of-day financial time series data._**

See Artificial Intelligence in Finance (ch. 04)  and `http://hilpisch.com/aiif_eikon_eod_data.csv`.

## Imports and Data

In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
pd.set_option("display.precision", 5)
np.set_printoptions(suppress=True,
        formatter={'float': lambda x: f'{x:.4f}'})

In [ ]:
raw = pd.read_csv('http://hilpisch.com/aiif_eikon_eod_data.csv',
                  index_col=0, parse_dates=True).dropna()

In [ ]:
rets = np.log(raw / raw.shift(1)).dropna()

## Portfolio Statistics

In [ ]:
def port_return_(rets_mean, weights):
    return np.dot(rets_mean, weights) * 252  # annualized

In [ ]:
def port_return(rets, weights):
    return np.dot(rets.mean(), weights) * 252  # annualized

In [ ]:
def port_volatility(rets, weights):
    return np.dot(weights, np.dot(rets.cov() * 252 , weights)) ** 0.5  # annualized

In [ ]:
def port_sharpe(rets, weights):
    return port_return(rets, weights) / port_volatility(rets, weights)

## Equal Weights

In [ ]:
symbols = rets.columns[:3]
symbols

In [ ]:
weights = len(symbols) * [1 / len(symbols)]  # reference portfolio (equal weights)
weights

### Out-of-Sample Testing

In [ ]:
res = pd.DataFrame()
for year in range(2010, 2019):
    rets_ = rets[symbols].loc[f'{year}-01-01':f'{year}-12-31']
    epv = port_volatility(rets_, weights)
    epr = port_return(rets_, weights)
    esr = epr / epv
    rets_ = rets[symbols].loc[f'{year + 1}-01-01':f'{year + 1}-12-31']
    rpv = port_volatility(rets_, weights)
    rpr = port_return(rets_, weights)
    rsr = rpr / rpv
    res = pd.concat([res, pd.DataFrame({'epv': epv, 'epr': epr, 'esr': esr,
                                   'rpv': rpv, 'rpr': rpr, 'rsr': rsr},
                                  index=[year + 1])])

In [ ]:
res

In [ ]:
res.mean()

In [ ]:
(res[['rpv', 'rpr', 'rsr']].mean().values -
 res[['epv', 'epr', 'esr']].mean().values)

In [ ]:
res[['epv', 'rpv']].corr()

In [ ]:
res[['epv', 'rpv']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Volatility');

In [ ]:
res[['epr', 'rpr']].corr()

In [ ]:
res[['epr', 'rpr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Return');

In [ ]:
res[['esr', 'rsr']].corr()

In [ ]:
res[['esr', 'rsr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Sharpe Ratio');

## Equal Returns

In [ ]:
from scipy.optimize import minimize

In [ ]:
bnds = len(symbols) * [(0, 1),]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}

In [ ]:
rets_mean = len(symbols) * [0.001]
rets_mean

### Optimal Weights

In [ ]:
opt_weights = {}
for year in range(2010, 2019):
    rets_ = rets[symbols].loc[f'{year - 1}-01-01':f'{year}-12-31']
    ow = minimize(lambda weights: -port_return_(rets_mean, weights) /
                                  port_volatility(rets_[symbols], weights),
                  len(symbols) * [1 / len(symbols)],
                  bounds=bnds,
                  constraints=cons)['x']
    opt_weights[year] = ow
ow = pd.DataFrame(opt_weights).T
ow.columns = symbols

In [ ]:
opt_weights

In [ ]:
ow.plot.bar(stacked=True, alpha=0.75);

In [ ]:
weights = len(symbols) * [1 / len(symbols)]  # reference portfolio (equal weights)

In [ ]:
((ow - weights) ** 2).mean(axis=1)

### Out-of-Sample Testing

In [ ]:
res = pd.DataFrame()
for year in range(2010, 2019):
    rets_ = rets[symbols].loc[f'{year}-01-01':f'{year}-12-31']
    epv = port_volatility(rets_, opt_weights[year])
    epr = port_return(rets_, opt_weights[year])
    esr = epr / epv
    rets_ = rets[symbols].loc[f'{year + 1}-01-01':f'{year + 1}-12-31']
    rpv = port_volatility(rets_, opt_weights[year])
    rpr = port_return(rets_, opt_weights[year])
    rsr = rpr / rpv
    res = pd.concat([res, pd.DataFrame({'epv': epv, 'epr': epr, 'esr': esr,
                                   'rpv': rpv, 'rpr': rpr, 'rsr': rsr},
                                  index=[year + 1])])

In [ ]:
res

In [ ]:
res.mean()

In [ ]:
(res[['rpv', 'rpr', 'rsr']].mean().values -
 res[['epv', 'epr', 'esr']].mean().values)

In [ ]:
res[['epv', 'rpv']].corr()

In [ ]:
res[['epv', 'rpv']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Volatility');

In [ ]:
res[['epr', 'rpr']].corr()

In [ ]:
res[['epr', 'rpr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Return');

In [ ]:
res[['esr', 'rsr']].corr()

In [ ]:
res[['esr', 'rsr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Sharpe Ratio');

## Equal Sharpe Ratio

In [ ]:
rets[symbols].mean() / rets[symbols].std()

In [ ]:
sr = rets['.SPX'].mean() / rets['.SPX'].std()  # daily

In [ ]:
sr

In [ ]:
rets_mean = sr * rets[symbols].std()
rets_mean

In [ ]:
rets[symbols].mean()

### Optimal Weights

In [ ]:
opt_weights = {}
exp_returns = {}
for year in range(2010, 2019):
    rets_ = rets[symbols].loc[f'{year - 1}-01-01':f'{year}-12-31']
    sr = (rets.loc[f'{year - 1}-01-01':f'{year}-12-31']['.SPX'].mean() /
         rets.loc[f'{year - 1}-01-01':f'{year}-12-31']['.SPX'].std())
    rets_mean = sr * rets_.std()
    ow = minimize(lambda weights: -port_return_(rets_mean, weights) /
                                  port_volatility(rets_, weights),
                  len(symbols) * [1 / len(symbols)],
                  bounds=bnds,
                  constraints=cons)['x']
    opt_weights[year] = ow
    exp_returns[year] = rets_mean
ow = pd.DataFrame(opt_weights).T
ow.columns = symbols

In [ ]:
opt_weights

In [ ]:
ow.plot.bar(stacked=True, alpha=0.75);

In [ ]:
weights = len(symbols) * [1 / len(symbols)]  # reference portfolio (equal weights)

In [ ]:
((ow - weights) ** 2).mean(axis=1)

### Out-of-Sample Testing

In [ ]:
res = pd.DataFrame()
for year in range(2010, 2019):
    rets_ = rets[symbols].loc[f'{year}-01-01':f'{year}-12-31']
    epv = port_volatility(rets_, opt_weights[year])
    epr = port_return_(exp_returns[year], opt_weights[year])
    esr = epr / epv
    rets_ = rets[symbols].loc[f'{year + 1}-01-01':f'{year + 1}-12-31']
    rpv = port_volatility(rets_, opt_weights[year])
    rpr = port_return(rets_, opt_weights[year])
    rsr = rpr / rpv
    res = pd.concat([res, pd.DataFrame({'epv': epv, 'epr': epr, 'esr': esr,
                                   'rpv': rpv, 'rpr': rpr, 'rsr': rsr},
                                  index=[year + 1])])

In [ ]:
res

In [ ]:
res.mean()

In [ ]:
(res[['rpv', 'rpr', 'rsr']].mean().values -
 res[['epv', 'epr', 'esr']].mean().values)

In [ ]:
res[['epv', 'rpv']].corr()

In [ ]:
res[['epv', 'rpv']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Volatility');

In [ ]:
res[['epr', 'rpr']].corr()

In [ ]:
res[['epr', 'rpr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Return');

In [ ]:
res[['esr', 'rsr']].corr()

In [ ]:
res[['esr', 'rsr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Sharpe Ratio');

## CAPM-Based Return Predictions 

In [ ]:
symbols = rets.columns[:3]
symbols

In [ ]:
symbols_ = list(symbols) + ['.SPX']
symbols_

In [ ]:
cov = rets[symbols_].cov() * 252
cov

In [ ]:
beta = cov['.SPX'][symbols] / cov['.SPX']['.SPX']
beta

In [ ]:
beta * rets['.SPX'].mean() * 252

### Optimal Weights

In [ ]:
bnds = len(symbols) * [(0, 1),]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}

In [ ]:
opt_weights = {}
exp_returns = {}
for year in range(2010, 2019):
    rets_ = rets[symbols_].loc[f'{year - 1}-01-01':f'{year}-12-31']
    cov = rets_.cov()
    beta = cov['.SPX'][symbols] / cov['.SPX']['.SPX']
    rets_mean = beta * rets_['.SPX'].mean()
    ow = minimize(lambda weights: -port_return_(rets_mean, weights) /
                                  port_volatility(rets_[symbols], weights),
                  len(symbols) * [1 / len(symbols)],
                  bounds=bnds,
                  constraints=cons)['x']
    opt_weights[year] = ow
    exp_returns[year] = rets_mean
ow = pd.DataFrame(opt_weights).T
ow.columns = symbols

In [ ]:
opt_weights

In [ ]:
ow.plot.bar(stacked=True, alpha=0.75);

In [ ]:
weights = len(symbols) * [1 / len(symbols)]  # reference portfolio (equal weights)

In [ ]:
((ow - weights) ** 2).mean(axis=1)

### Out-of-Sample Testing

In [ ]:
res = pd.DataFrame()
for year in range(2010, 2019):
    rets_ = rets[symbols].loc[f'{year}-01-01':f'{year}-12-31']
    epv = port_volatility(rets_, opt_weights[year])
    epr = port_return_(exp_returns[year], opt_weights[year])
    esr = epr / epv
    rets_ = rets[symbols].loc[f'{year + 1}-01-01':f'{year + 1}-12-31']
    rpv = port_volatility(rets_, opt_weights[year])
    rpr = port_return(rets_, opt_weights[year])
    rsr = rpr / rpv
    res = pd.concat([res, pd.DataFrame({'epv': epv, 'epr': epr, 'esr': esr,
                                   'rpv': rpv, 'rpr': rpr, 'rsr': rsr},
                                  index=[year + 1])])

In [ ]:
res

In [ ]:
res.mean()

In [ ]:
(res[['rpv', 'rpr', 'rsr']].mean().values -
 res[['epv', 'epr', 'esr']].mean().values)

In [ ]:
res[['epv', 'rpv']].corr()

In [ ]:
res[['epv', 'rpv']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Volatility');

In [ ]:
res[['epr', 'rpr']].corr()

In [ ]:
res[['epr', 'rpr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Return');

In [ ]:
res[['esr', 'rsr']].corr()

In [ ]:
res[['esr', 'rsr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Sharpe Ratio');

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>